In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: pho
"""
%load_ext autoreload
%autoreload 2
import sys
import importlib
from threading import Thread
import time # for time.sleep
from ipygany import PolyMesh, Scene, IsoColor, WarpByScalar
import pyvista as pv
# from pyvista import _vtk
import pyvistaqt as pvqt
import colorcet as cc # Colormaps:
import numpy as np
import h5py
import hdf5storage # conda install hdf5storage
from pathlib import Path
import bqplot.scales
import seaborn as sns
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
# import mplcursors
import math # For color map generation
from matplotlib.colors import ListedColormap
from matplotlib.cm import hsv

import ipywidgets as widgets
# from PyQt5 import QtWidgets, uic
from pyvistaqt import QtInteractor, MainWindow
# from pyqt6 import QApplication
from IPython.external.qt_for_kernel import QtGui
from PyQt5.QtWidgets import QApplication

import datetime as dt
import panel as pn # for interactive widgets
pn.extension()

In [2]:
# import PhoPositionalData as pdp
# from PhoPositionalData import load_exported, process_data
from PhoPositionalData.load_exported import *
# from PhoPositionalData.process_data import process_positionalAnalysis_data, gen_2d_histrogram, get_heatmap_color_vectors, process_chunk_equal_poritions_data, extract_spike_timeseries
from PhoPositionalData.process_data import *
from PhoPositionalData.plot_data import *
from PhoPositionalData.plotting.animations import * # make_mp4_from_plotter
from PhoPositionalData.import_data import * # build_spike_positions_list, build_cellID_reverse_lookup_map
from PhoPositionalData.analysis.interactive_placeCell_config import InteractivePlaceCellConfig, VideoOutputModeConfig, PlottingConfig, PlacefieldComputationParameters  # VideoOutputModeConfig, InteractivePlaceCellConfigs
from PhoPositionalData.analysis.interactive_placeCell_config import print_subsession_neuron_differences

In [3]:
# NeuroPy (Diba Lab Python Repo) Loading
try:
    from neuropy import core
    importlib.reload(core)
except ImportError:
    sys.path.append(r'C:\Users\Pho\repos\NeuroPy') # Windows
    # sys.path.append('/home/pho/repo/BapunAnalysis2021/NeuroPy') # Linux
    # sys.path.append(r'/Users/pho/repo/Python Projects/NeuroPy') # MacOS
    print('neuropy module not found, adding directory to sys.path. \n >> Updated sys.path.')
    from neuropy import core
from neuropy.core.dataSession import SessionConfig, DataSessionLoader, DataSession, processDataSession
from neuropy.core.epoch import NamedEpoch

## Data must be pre-processed using the MATLAB script located here: 
# R:\data\KDIBA\gor01\one\IIDataMat_Export_ToPython_2021_11_23.m

# From pre-computed .mat files:
## 07: 
basedir = r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53'

# ## 08:
# basedir = r'R:\data\KDIBA\gor01\one\2006-6-08_14-26-15'

neuropy module not found, adding directory to sys.path. 
 >> Updated sys.path.


In [4]:
# KDiba Old Format:
sess = DataSessionLoader.kdiba_old_format_session(r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53')
active_sess_config = sess.config
session_name = sess.name

	 basepath: R:\data\KDIBA\gor01\one\2006-6-07_11-26-53
	 session_name: 2006-6-07_11-26-53
Loading matlab import file: R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.spikeII.mat...
done.
Loading matlab import file: R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.epochs_info.mat...
done.
Loading matlab import file: R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.position_info.mat...
done.
computing linear positions for all active epochs for session...
Saving updated position results to R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.position.npy...
2006-6-07_11-26-53.position.npy saved
done.

session.position linear positions computed!


In [5]:
sess.epochs.to_dataframe()

,start,stop,label,duration
0,0.000000,1739.153364,maze1,1739.153364
1,1739.153364,1932.420005,maze2,193.266641


In [21]:
## NOTE: The only place that I need to be careful with indexing is with sess.position properties, as these appear to be represented in terms of the number of 60Hz samples instead of in seconds like the Neurons and other classes.
active_epoch_name = 'maze1'
# active_epoch_name = 'maze2'

active_epoch = NamedEpoch(
        name=active_epoch_name,
        start_end_times=sess.epochs[active_epoch_name]
    )

# active_epoch_name = 'maze'
# active_epoch = NamedEpoch(
#         name=active_epoch_name,
#         start_end_times=[sess.epochs['maze1'][0], sess.epochs['maze2'][1]]
#     )
# Constraining to epoch with times (start: 0.0, end: 1932.4200048116618)
# 20996/932598 total spikes spanning 40/64 units remain in subsession

active_subplots_shape = (1,1) # Single subplot
# active_subplots_shape = '1|2' # 1 subplot on left, two on right

## Get the config corresponding to this epoch/session settings:
# active_config = InteractivePlaceCellConfig(active_session_config=active_sess_config, active_epochs=active_epoch)
active_config = InteractivePlaceCellConfig(active_session_config=active_sess_config, active_epochs=active_epoch,
                        video_output_config=VideoOutputModeConfig(active_frame_range=np.arange(11070.0, 13970.0), video_output_parent_dir=Path('output', active_sess_config.session_name, active_epoch.name), active_is_video_output_mode=False),
                        plotting_config=PlottingConfig(output_subplots_shape=active_subplots_shape, output_parent_dir=Path('output', active_sess_config.session_name, active_epoch.name))) # '3|1    
## Looks like the only real outputs are: active_epoch_session_Neurons, active_epoch_pos
active_config.plotting_config.active_output_parent_dir.mkdir(parents=True, exist_ok=True) # makes the directory if it isn't already there

# Get the filtered and time-constrainted session and position data for this epoch:
active_epoch_session = sess.get_neuron_type('pyramidal').time_slice(active_epoch.t_start, active_epoch.t_stop)
print_subsession_neuron_differences(sess.neurons, active_epoch_session.neurons)
active_epoch_session_Neurons, active_epoch_pos, active_epoch_position_times = active_epoch_session.neurons, active_epoch_session.position, active_epoch_session.position.time

Constraining to units with type: pyramidal
Constraining to epoch with times (start: 0.0, end: 1739.1533641185379)
18473/932598 total spikes spanning 40/64 units remain in subsession


In [22]:
active_epoch_session.spikes_df
# Get only the rows with a lap != -1:
# spikes_df = spikes_df[(spikes_df.lap != -1)] # 229887 rows × 13 columns
# spikes_df # 932598 rows × 14 columns

,t,shank,cluster,aclu,qclu,cluinfo,x,y,speed,traj,lap,cell_type,t_seconds,flat_spike_idx
1,161,8,13,53,4,"(0, 0, 4, 4)",NaN,NaN,NaN,0,-1,NeuronType.PYRAMIDAL,0.0161,1
37,2723,7,10,45,4,"(0, 0, 4, 0)",0.374502,0.338421,0.000000,0,-1,NeuronType.PYRAMIDAL,0.2723,37
38,2830,7,10,45,4,"(0, 0, 4, 0)",0.374502,0.338421,0.000000,0,-1,NeuronType.PYRAMIDAL,0.2830,38
42,2940,7,10,45,4,"(0, 0, 4, 0)",0.374502,0.338421,0.000000,0,-1,NeuronType.PYRAMIDAL,0.2940,42
51,3696,7,10,45,4,"(0, 0, 4, 0)",0.374502,0.338421,0.000000,0,-1,NeuronType.PYRAMIDAL,0.3696,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268637,17386079,10,3,57,2,"(0, 0, 1, 3)",0.868184,0.476945,4.852559,13,-1,NeuronType.PYRAMIDAL,1738.6079,268637
268690,17390228,7,10,45,4,"(0, 0, 4, 0)",0.866364,0.475073,1.142863,13,-1,NeuronType.PYRAMIDAL,1739.0228,268690
268692,17390449,7,10,45,4,"(0, 0, 4, 0)",0.866335,0.475049,0.834770,13,-1,NeuronType.PYRAMIDAL,1739.0449,268692
268695,17390532,8,13,53,4,"(0, 0, 4, 4)",0.866324,0.475039,0.719106,13,-1,NeuronType.PYRAMIDAL,1739.0532,268695


In [24]:
active_epoch_session.position.to_dataframe()

,t,x,lin_pos,y,speed
0,0.000000,103.773990,79.381304,100.775778,0.000000
1,0.033368,103.274434,79.554667,100.510469,16.952002
2,0.066735,102.699105,79.849525,100.169432,20.044140
3,0.100103,102.233116,80.193347,99.828504,17.304203
4,0.133470,101.923169,80.529185,99.513796,13.237968
...,...,...,...,...,...
52116,1738.985881,211.882873,-29.457400,138.599074,15.460512
52117,1739.019249,211.383281,-28.979510,138.712202,15.351718
52118,1739.052616,210.881820,-28.547771,138.825754,15.409162
52119,1739.085984,210.375746,-28.209280,138.940351,15.550918


In [ ]:
# # Group by the lap column:
# lap_grouped_spikes_df = spikes_df.groupby(['lap']) #  as_index=False keeps the original index
# lap_spike_indicies = list()
# for i in np.arange(sess.laps.n_laps):
#     curr_lap_id = sess.laps.lap_id[i]
#     #curr_flat_cell_indicies = (flat_spikes_out_dict['aclu'] == curr_cell_id) # the indicies where the cell_id matches the current one
#     curr_lap_dataframe = lap_grouped_spikes_df.get_group(curr_lap_id)
#     # curr_lap_spike_indicies = curr_lap_dataframe.flat_spike_idx.values
#     lap_spike_indicies.append(curr_lap_dataframe.flat_spike_idx.values)
#     # spiketrains.append(curr_cell_dataframe[time_variable_name].to_numpy())
#     # shank_ids[i] = curr_cell_dataframe['shank'].to_numpy()[0] # get the first shank identifier, which should be the same for all of this curr_cell_id
#     # cell_quality[i] = curr_cell_dataframe['qclu'].mean() # should be the same for all instances of curr_cell_id, but use mean just to make sure
#     # cell_type.append(curr_cell_dataframe['cell_type'].to_numpy()[0])

# print('lap_spike_indicies: {}'.format(lap_spike_indicies))
# # sess.laps.n_laps

In [25]:
## Compute Placefields if needed:
from neuropy.analyses import Pf1D, Pf2D, perform_compute_placefields, plot_all_placefields
from neuropy.plotting.spikes import get_neuron_colors

should_force_recompute_placefields = True
should_display_2D_plots = False
# Setup the placefield computation config
active_config.computation_config = PlacefieldComputationParameters(speed_thresh=3, grid_bin=2, smooth=2)
# print_subsession_neuron_differences(sess.neurons, active_epoch_session_Neurons)

try: active_epoch_placefields1D
except NameError: active_epoch_placefields1D = None # Checks variable active_epoch_placefields's existance, and sets its value to None if it doesn't exist so it can be checked in the next step

try: active_epoch_placefields2D
except NameError: active_epoch_placefields2D = None # Checks variable active_epoch_placefields's existance, and sets its value to None if it doesn't exist so it can be checked in the next step

active_epoch_placefields1D, active_epoch_placefields2D = perform_compute_placefields(active_epoch_session_Neurons, active_epoch_pos, active_config.computation_config, active_epoch_placefields1D, active_epoch_placefields2D, should_force_recompute_placefields=True)
# Plot the placefields computed and save them out to files:
if should_display_2D_plots:
    ax_pf_1D, occupancy_fig, active_pf_2D_figures = plot_all_placefields(active_epoch_placefields1D, active_epoch_placefields2D, active_config)
else:
    print('skipping 2D placefield plots')
    
# Focus on the 2D placefields:
active_epoch_placefields = active_epoch_placefields2D
# Get the cell IDs that have a good place field mapping:
good_placefield_neuronIDs = np.array(active_epoch_placefields.ratemap.neuron_ids) # in order of ascending ID
print('good_placefield_neuronIDs: {}; ({} good)'.format(good_placefield_neuronIDs, len(good_placefield_neuronIDs)))

## Filter by neurons with good placefields only:
active_epoch_session = active_epoch_session.get_by_id(good_placefield_neuronIDs) # Filter by good placefields only, and this fetch also ensures they're returned in the order of sorted ascending index ([ 2  3  5  7  9 12 18 21 22 23 26 27 29 34 38 45 48 53 57])
active_epoch_session_Neurons = active_epoch_session.neurons

## Plotting Colors:
pf_sort_ind = np.array([int(i) for i in np.arange(len(active_epoch_session.neuron_ids))]) # convert to integer scalar array
pf_colors = get_neuron_colors(pf_sort_ind) # colors are by ascending index ID

active_config.plotting_config.pf_sort_ind = pf_sort_ind
active_config.plotting_config.pf_colors = pf_colors
active_config.plotting_config.active_cells_colormap = pf_colors.T # Make the colormap from the listed colors, used seemingly only by 'runAnalysis_PCAandICA(...)'
active_config.plotting_config.active_cells_listed_colormap = ListedColormap(active_config.plotting_config.active_cells_colormap)


Recomputing active_epoch_placefields...
	 done.
Recomputing active_epoch_placefields2D...
	 done.
skipping 2D placefield plots
good_placefield_neuronIDs: [ 2  3  4  5  7  9 11 12 17 18 21 22 23 24 27 28 29 33 34 38 39 42 44 45
 46 47 48 53 55 57 58]; (31 good)


In [26]:
# flattened_spikes.flattened_spike_identities # array([53, 45, 45, ..., 45, 22,  2])
# flattened_spikes.flattened_sort_indicies # array([16984, 13674, 13675, ..., 16068,  7575,   580], dtype=int64)
# flattened_spikes.flattened_spike_times # array([1.6100000e-02, 2.7230000e-01, 2.8300000e-01, ..., 1.9308899e+03, 1.9316915e+03, 1.9320706e+03])
# print(flattened_spike_positions_list) # [[103.5329515  100.94484811 100.86902587 ... 215.55358368 216.0612899 217.59276516], [100.64776532  98.26058916  98.16262888 ... 139.36778275 138.94465291 139.07649496]]
# num_flattened_spikes # 19647

In [27]:
## ICA and PCA Analysis
should_show_2D_ICA_plots = False
from PhoPositionalData.analysis.neuronal_dimensionality_reduction import runAnalysis_PCAandICA
active_session_ensembles, template, zsc_template, pca_data = runAnalysis_PCAandICA(active_epoch_session_Neurons, bin_size=0.250, frate_thresh=0.1, should_plot=should_show_2D_ICA_plots, active_cells_colormap=active_config.plotting_config.active_cells_colormap)

Based on frate_thresh, excluded neuron_ids: [ 4  7 11 33 47]


## Main Spike/Placemap plotting:

In [51]:
# from PhoPositionalData.plotting.gui import customize_default_pyvista_theme, print_controls_helper_text
# customize_default_pyvista_theme() # Sets the default theme values to those specified in my imported file
# # This defines the position of the vertical/horizontal splitting, in this case 40% of the vertical/horizontal dimension of the window
# # pv.global_theme.multi_rendering_splitting_position = 0.40
# pv.global_theme.multi_rendering_splitting_position = 0.80

# from PhoPositionalData.plotting.camera_manipulation import * # apply_camera_view, apply_close_perspective_camera_view, apply_close_overhead_zoomed_camera_view
# from PhoPositionalData.plotting.spikeAndPositions import build_active_spikes_plot_data, build_flat_map_plot_data, perform_plot_flat_arena, build_spike_spawn_effect_light_actor, spike_geom_circle, spike_geom_box, spike_geom_cone, animal_location_circle, animal_location_trail_circle
# # from PhoPositionalData.plotting.spikeAndPositions import InteractiveSliderWrapper # for wrapping the slider

# # num_time_points = active_epoch_pos.n_frames / curr_plot_update_step
# # print('num_time_points: {}\n'.format(num_time_points))

t = active_epoch_pos.time
x = active_epoch_pos.x
y = active_epoch_pos.y

# Plot Placemaps on 3D Plot

In [52]:
from PhoGui.InteractivePlotter.InteractivePlaceCellTuningCurvesDataExplorer import InteractivePlaceCellTuningCurvesDataExplorer

try: pActiveTuningCurvesPlotter
except NameError: pActiveTuningCurvesPlotter = None # Checks variable p's existance, and sets its value to None if it doesn't exist so it can be checked in the next step
ipcDataExplorer = InteractivePlaceCellTuningCurvesDataExplorer(active_config, x, y, active_epoch_placefields, pf_colors, extant_plotter=pActiveTuningCurvesPlotter)

In [53]:
pActiveTuningCurvesPlotter = ipcDataExplorer.plot(pActiveTuningCurvesPlotter)

No extant BackgroundPlotter
Creating a new BackgroundPlotter
done.


C:\Users\Pho\repos\NeuroPy\neuropy\utils\mathutil.py:23: RuntimeWarning: invalid value encountered in true_divide
  return (x - np.min(x, axis=axis, keepdims=True)) / np.ptp(


num_curr_tuning_curves: 31


In [54]:
## Slider with Callback Function Example:
import PhoGui
from PhoGui.InteractivePlotter.PhoInteractivePlotter import PhoInteractivePlotter
from PhoGui.InteractivePlotter.shared_helpers import InteractivePyvistaPlotterBuildIfNeededMixin
from PhoGui.InteractivePlotter.InteractivePlaceCellDataExplorer import InteractivePlaceCellDataExplorer

active_config.plotting_config.show_legend = True

try: pActiveInteractivePlaceSpikesPlotter
except NameError: pActiveInteractivePlaceSpikesPlotter = None # Checks variable p's existance, and sets its value to None if it doesn't exist so it can be checked in the next step
ipspikesDataExplorer = InteractivePlaceCellDataExplorer(active_config, active_epoch_session, extant_plotter=pActiveInteractivePlaceSpikesPlotter)
print('done.')


 flattened_spike_positions_list_new: (2, 17449), [[       nan 0.37450201 0.37450201 ... 0.86633532 0.86632449 0.86632266]
 [       nan 0.33842111 0.33842111 ... 0.47504852 0.47503917 0.47503759]]

 flattened_spike_positions_list_old: (2, 17449), [[103.53295196 100.94485182 100.86902972 ... 210.99778204 210.87296572
  210.85173243]
 [100.64776556  98.26059394  98.16263385 ... 138.79949559 138.82775922
  138.83256733]]


active_epoch_pos.sampling_rate (Hz): 29.96976599553983
longer_spikes_window - curr_view_window_length_samples - 30689
recent_spikes_window - curr_view_window_length_samples - 299
Applying custom Pyvista theme.
done.
done.


In [40]:
pActiveInteractivePlaceSpikesPlotter = ipspikesDataExplorer.plot(pActivePlotter=pActiveInteractivePlaceSpikesPlotter)

No open BackgroundPlotter
Creating a new BackgroundPlotter
done.
[f] - Focus and zoom in on the last clicked point
shift+click - Drag to pan the rendering scene
ctrl+click - Rotate the scene in 2D


In [ ]:

# ## Playback animation:
# # pl = PhoInteractivePlotter(pyvista_plotter=p)

# # ## Testing Updating the Timestamp Slider Programmatically:
# # interactive_timestamp_slider_wrapper.curr_index
# # interactive_timestamp_slider_wrapper.step_index(15)
        
# animate = InterfaceProperties(interactive_timestamp_slider_wrapper)
# # An unused constant-time callback that calls back every so often to perform updates
# p.add_callback(animate, interval=16)  # to be smooth on 60Hz

# # A checkbox that decides whether we're playing back at a constant rate or not.
# interactive_checkbox_actor = p.add_checkbox_button_widget(toggle_animation, value=False, color_on='green')